In [33]:
%matplotlib inline

In [34]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.utils.data as data
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [35]:
SOS_token = 0
EOS_token = 1
PAD_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: 'PAD'}
        self.n_words = 3  # Count SOS , EOS and PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1



In [36]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s



In [37]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [38]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]



In [39]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4346
eng 2804
['nous sommes plutot occupes ici .', 'we re kind of busy here .']


In [40]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        print(torch.eye(hidden_size).size())
        print(self.embedding.weight.data.size())

        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input,batch_size, hidden):
        embedded = self.embedding(input).view(1, batch_size, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self,batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)


In [41]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)

        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input,batch_size, hidden):
        output = self.embedding(input).view(1, batch_size, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self,batch_size):
        return torch.zeros(1, batch_size, self.hidden_size, device=device)




In [42]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence, max_length):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    indexes.extend([PAD_token] * (max_length - len(indexes)))
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPairs(input_lang, output_lang, pairs, max_length):
    res = []
    for pair in pairs:
        input_variable = tensorFromSentence(input_lang, pair[0], max_length)
        target_variable = tensorFromSentence(output_lang, pair[1], max_length)
        res.append((input_variable, target_variable))
    return res

In [43]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    batch_size = input_tensor.size(0)
    encoder_hidden = encoder.initHidden(batch_size)

    input_tensor = input_tensor.transpose(0, 1)
    target_tensor = target_tensor.transpose(0, 1)

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, batch_size, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder( input_tensor[ei], batch_size, encoder_hidden)
        encoder_outputs[ei] = encoder_output[0]
    ## [[SOS_token]* batch_size]
    decoder_input = torch.tensor([SOS_token]* batch_size, device=device)

    decoder_hidden = encoder_hidden

    # use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    use_teacher_forcing = True
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(  decoder_input, batch_size, decoder_hidden)

            for do,tt in zip(decoder_output, target_tensor[di]):
                loss += criterion(do.view(1,-1),tt)

            # loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder( decoder_input, batch_size, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [44]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [45]:
def trainIters(encoder, decoder, n_iters,train_loader, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(filter(lambda x: x.requires_grad, encoder.parameters()),lr=learning_rate)
    decoder_optimizer = optim.SGD(filter(lambda x: x.requires_grad, decoder.parameters()),lr=learning_rate)
    # encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        print("iter",iter)
        batch_size=train_loader.batch_size
        for batch_x, batch_y in train_loader:
            loss = train(batch_x, batch_y, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            print_loss_total += loss
            plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every/batch_size
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every/batch_size
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [46]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [47]:
def evaluate(encoder, decoder, loader, max_length=MAX_LENGTH):
    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_size = batch_x.size(0)
            encoder_hidden = encoder.initHidden(batch_size)


            input_tensor = batch_x.transpose(0, 1)
            target_tensor = batch_y.transpose(0, 1)
            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            output = torch.empty(target_length,batch_size, dtype=torch.long, device=device)


            encoder_outputs = torch.zeros(max_length,batch_size, encoder.hidden_size, device=device)

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei], batch_size, encoder_hidden)
                encoder_outputs[ei] += encoder_output[0, 0]
            decoder_input = torch.tensor([SOS_token]*batch_size, device=device)  # SOS

            decoder_hidden = encoder_hidden

            decoded_sentences = []


            for di in range(target_length):
                decoder_output, decoder_hidden= decoder(decoder_input, batch_size, decoder_hidden)
                topv, topi = decoder_output.data.topk(1)
                output[di] = topi.squeeze()
                decoder_input = topi.squeeze().detach()
            output = output.transpose(0,1)
            for di in range(target_length):
                decoded_sentences.append([word for word in output[di]])

            # show output
            for index,[ip,op] in enumerate(zip(batch_x, batch_y)):
                # pair=one sentence
                input_sentence=' '.join([input_lang.index2word[word.item()] for word in ip])
                print('>', input_sentence)
                real_sentence=' '.join([output_lang.index2word[word.item()] for word in op])
                print('=', real_sentence)

                output_sentence = ' '.join([output_lang.index2word[word.item()] for word in decoded_sentences[index]])
                print('<', output_sentence)
                print('')



In [48]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

torch.Size([256, 256])
torch.Size([4346, 256])


In [49]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
pairs = tensorsFromPairs(input_lang, output_lang, pairs, MAX_LENGTH)
batch_size = 64
train_num = int(len(pairs)*0.9)
train_loader = data.DataLoader(pairs[:train_num], batch_size=batch_size, shuffle=True)
                            # pairs[train_num:]
test_loader = data.DataLoader(pairs[train_num:train_num+batch_size], batch_size=batch_size, shuffle=True)

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4346
eng 2804


In [53]:
trainIters(encoder1, attn_decoder1, 200,train_loader, print_every=1)

evaluate(encoder1, attn_decoder1,test_loader)



iter 1
0m 16s (- 551m 7s) (1 0%) 979.7561
iter 2
0m 32s (- 543m 27s) (2 0%) 973.4772
iter 3
0m 49s (- 544m 58s) (3 0%) 1023.9541
iter 4
1m 5s (- 542m 57s) (4 0%) 905.9423
iter 5
1m 21s (- 542m 53s) (5 0%) 905.9676
iter 6
1m 37s (- 541m 28s) (6 0%) 928.8838
iter 7
1m 54s (- 541m 11s) (7 0%) 892.9701
iter 8
2m 10s (- 540m 0s) (8 0%) 949.3112
iter 9
2m 26s (- 538m 52s) (9 0%) 1006.5703
iter 10
2m 42s (- 538m 9s) (10 0%) 841.2462
iter 11
2m 58s (- 539m 14s) (11 0%) 895.1962
iter 12
3m 15s (- 538m 43s) (12 0%) 958.4571
iter 13
3m 31s (- 538m 1s) (13 0%) 857.6532
iter 14
3m 47s (- 538m 6s) (14 0%) 895.5060
iter 15
4m 3s (- 537m 53s) (15 0%) 849.4040
iter 16
4m 20s (- 537m 24s) (16 0%) 834.1148
iter 17
4m 36s (- 537m 18s) (17 0%) 912.2985
iter 18
4m 52s (- 536m 47s) (18 0%) 837.8807
iter 19
5m 8s (- 536m 45s) (19 0%) 810.3010
iter 20
5m 25s (- 536m 27s) (20 1%) 841.3601
iter 21
5m 41s (- 536m 6s) (21 1%) 838.4370
iter 22
5m 57s (- 535m 45s) (22 1%) 868.3809
iter 23
6m 13s (- 535m 31s) (23 1%)

Locator attempting to generate 1768 ticks ([47.6, ..., 401.00000000000006]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1768 ticks ([47.6, ..., 401.00000000000006]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1768 ticks ([47.6, ..., 401.00000000000006]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1768 ticks ([47.6, ..., 401.00000000000006]), which exceeds Locator.MAXTICKS (1000).
